In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from openpyxl import load_workbook
from openpyxl.drawing.image import Image

# --- Step 1: Read Excel files ---
super_df = pd.read_excel("/Users/judycheng/Desktop/supercharger in washington state.xls")
residents_df = pd.read_excel("/Users/judycheng/Desktop/Population 2024 age 25 to 59.xlsx")
ev_df = pd.read_excel("/Users/judycheng/Desktop/coordinates_output.xlsm")

# --- Step 2: King County data ---
king_super = super_df[(super_df["State"] == "Washington") & (super_df["County"] == "King County")]
king_residents = residents_df[residents_df["County"] == "King County"]
king_ev = ev_df[(ev_df["State"] == "WA") & (ev_df["County"] == "King")]

population_column = "total"
population_25_59 = king_residents[population_column].values[0]
num_chargers = len(king_super)
num_evs = len(king_ev)

print(f"King County population (25-59): {population_25_59}")
print(f"Existing Superchargers: {num_chargers}")
print(f"Existing EVs: {num_evs}")

# --- Step 3: Policy targets ---
policy_targets = {2030: 0.45, 2035: 0.60, 2040: 0.70, 2050: 0.95}

# --- Step 4: Lower and Upper Bound charger targets ---
king_county_area = 2307  # sq mi
radius_miles = 2
area_per_charger = np.pi * radius_miles**2
lower_chargers = int(np.ceil(king_county_area / area_per_charger))  # geo coverage (2-mile radius)

upper_base_target = population_25_59 / 1500  # 1 per 1500 residents (baseline upper bound)

print(f"Lower bound chargers (geo coverage): {lower_chargers}")
print(f"Original upper bound (1/1500 residents): {upper_base_target:.0f}")

# --- Step 5: Build scenarios ---
years = list(range(2025, 2051))
results = []

# --- Lower Bound (constant build per year) ---
current_lower = num_chargers
total_lower_needed = lower_chargers - num_chargers
n_years = len(years)
yearly_build_lower = int(round(total_lower_needed / n_years))

for y in years:
    new_lower = yearly_build_lower
    if current_lower + new_lower > lower_chargers:
        new_lower = lower_chargers - current_lower
    current_lower += new_lower

    # EV adoption
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 10 * (y - 2025) * 0.5
    elif y <= 2035:
        start_rate = 0.10 + (policy_targets[2030] - 0.10) * 0.5
        effective_adoption = start_rate + (policy_targets[2035] - start_rate) / 5 * (y - 2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Lower Bound (2-mile coverage, constant build)",
        "Year": y,
        "Total_Chargers": int(current_lower),
        "New_Chargers": new_lower,
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

# --- Upper Bound (3-phase slowdown, nearest whole number) ---
base_build_rate = upper_base_target / n_years
current_upper = num_chargers

for y in years:
    if 2025 <= y <= 2030:
        yearly_add = int(round(base_build_rate * 1.0))
    elif 2031 <= y <= 2040:
        yearly_add = int(round(base_build_rate * 0.6))
    elif 2041 <= y <= 2050:
        yearly_add = int(round(base_build_rate * 0.2))
    else:
        yearly_add = 0

    current_upper += yearly_add

    # EV adoption (upper bound)
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 5 * (y - 2025)
    elif y <= 2035:
        effective_adoption = policy_targets[2030] + (policy_targets[2035] - policy_targets[2030]) / 5 * (y - 2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Upper Bound (3-phase slowdown)",
        "Year": y,
        "Total_Chargers": int(current_upper),
        "New_Chargers": yearly_add,
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

forecast_df = pd.DataFrame(results)

# --- Step 6: Save results to Excel ---
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "king_county_ev_projection_constant_lower.xlsx")
with pd.ExcelWriter(desktop_path, engine="openpyxl") as writer:
    forecast_df.to_excel(writer, sheet_name="Forecast", index=False)

# --- Step 7: Charts ---
# Total Chargers
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)
plt.axvspan(2025, 2030, color="green", alpha=0.1, label="Phase 1")
plt.axvspan(2030, 2040, color="yellow", alpha=0.1, label="Phase 2")
plt.axvspan(2040, 2050, color="orange", alpha=0.1, label="Phase 3")
plt.xlabel("Year")
plt.ylabel("Total Chargers")
plt.title("King County: EV Charger Projection (Lower Constant vs 3-Phase Upper)")
plt.legend()
plt.grid(True)
plt.tight_layout()
charger_chart = os.path.join(os.path.expanduser("~"), "Desktop", "charger_projection_constant_lower.png")
plt.savefig(charger_chart)
plt.close()

# EV Adoption Rate
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Adoption_Rate"], marker="o", label=sc)
plt.xlabel("Year")
plt.ylabel("EV Adoption Rate")
plt.title("King County: EV Adoption Rate Forecast (2025–2050)")
plt.legend()
plt.grid(True)
plt.tight_layout()
adoption_chart = os.path.join(os.path.expanduser("~"), "Desktop", "ev_adoption_rate_constant_lower.png")
plt.savefig(adoption_chart)
plt.close()

# --- Step 8: Embed charts in Excel ---
wb = load_workbook(desktop_path)
ws = wb.create_sheet(title="Charts")
ws.add_image(Image(charger_chart), "A1")
ws.add_image(Image(adoption_chart), "A40")
wb.save(desktop_path)

print(f"\n✅ Projection complete. Excel and charts saved to: {desktop_path}")
print(f"Final 2050 upper bound chargers: {int(current_upper)}")


King County population (25-59): 1241805
Existing Superchargers: 13
Existing EVs: 101838
Lower bound chargers (geo coverage): 184
Original upper bound (1/1500 residents): 828

✅ Projection complete. Excel and charts saved to: /Users/judycheng/Desktop/king_county_ev_projection_constant_lower.xlsx
Final 2050 upper bound chargers: 455


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math
from openpyxl import load_workbook
from openpyxl.drawing.image import Image

# --- Step 1: Read Excel files ---
super_df = pd.read_excel("/Users/judycheng/Desktop/supercharger in washington state.xls")
residents_df = pd.read_excel("/Users/judycheng/Desktop/Population 2024 age 25 to 59.xlsx")
ev_df = pd.read_excel("/Users/judycheng/Desktop/coordinates_output.xlsm")

# --- Step 2: Pierce County data ---
pierce_super = super_df[(super_df["State"] == "Washington") & (super_df["County"] == "Pierce County")]
pierce_residents = residents_df[residents_df["County"] == "Pierce County"]
pierce_ev = ev_df[(ev_df["State"] == "WA") & (ev_df["County"] == "Pierce")]

population_column = "total"
population_25_59 = pierce_residents[population_column].values[0]
num_chargers = len(pierce_super)
num_evs = len(pierce_ev)

print(f"Pierce County population (25-59): {population_25_59}")
print(f"Existing Superchargers (from Excel): {num_chargers}")
print(f"Existing EVs: {num_evs}")

# --- Step 3: Policy targets ---
policy_targets = {2030: 0.45, 2035: 0.60, 2040: 0.70, 2050: 0.95}

# --- Step 4: Lower and Upper Bound charger targets ---
pierce_county_area = 1790  # sq mi
radius_miles = 3
area_per_charger = np.pi * radius_miles**2  # 28.27 sq mi per charger
lower_chargers = int(np.ceil(pierce_county_area / area_per_charger))  # geo coverage (3-mile radius)

upper_base_target = population_25_59 / 2500  # 1 per 2500 residents

print(f"Lower bound chargers (geo coverage, 3-mile radius): {lower_chargers}")
print(f"Original upper bound (1/2500 residents): {upper_base_target:.0f}")

# --- Step 5: Build scenarios ---
years = list(range(2025, 2051))
results = []

# --- LOWER BOUND (constant build per year, rounded up) ---
current_lower = num_chargers
total_lower_needed = max(0, lower_chargers - num_chargers)
n_years = len(years)
yearly_build_lower_exact = total_lower_needed / n_years  # keep as float

print("\n--- LOWER BOUND DEBUG ---")
print(f"Existing chargers (from Excel): {num_chargers}")
print(f"Target chargers (3-mile coverage): {lower_chargers}")
print(f"Total additional chargers needed: {total_lower_needed}")
print(f"Exact average build per year: {yearly_build_lower_exact:.2f}")
print("---------------------------\n")

for i, y in enumerate(years):
    # Round up build to nearest whole number
    new_lower = math.ceil(yearly_build_lower_exact)

    # Adjust last year so total hits exactly target
    if current_lower + new_lower > lower_chargers or i == len(years) - 1:
        new_lower = lower_chargers - current_lower

    current_lower += new_lower

    # EV adoption logic
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 10 * (y - 2025) * 0.5
    elif y <= 2035:
        start_rate = 0.10 + (policy_targets[2030] - 0.10) * 0.5
        effective_adoption = start_rate + (policy_targets[2035] - start_rate) / 5 * (y - 2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Lower Bound (3-mile coverage, constant build)",
        "Year": y,
        "Total_Chargers": int(current_lower),
        "New_Chargers": int(new_lower),
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

print(f"✅ Lower-bound chargers total at 2050: {int(current_lower)} (target: {lower_chargers})\n")

# Preview early years for verification
debug_df = pd.DataFrame([r for r in results if r["Scenario"].startswith("Lower Bound")])
print(debug_df.head(10)[["Year", "Total_Chargers", "New_Chargers"]])

# --- UPPER BOUND (3-phase slowdown) ---
base_build_rate = upper_base_target / n_years
current_upper = num_chargers

for y in years:
    if 2025 <= y <= 2030:
        yearly_add = int(round(base_build_rate * 1.0))
    elif 2031 <= y <= 2040:
        yearly_add = int(round(base_build_rate * 0.6))
    elif 2041 <= y <= 2050:
        yearly_add = int(round(base_build_rate * 0.2))
    else:
        yearly_add = 0

    current_upper += yearly_add

    # EV adoption (upper)
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 5 * (y - 2025)
    elif y <= 2035:
        effective_adoption = policy_targets[2030] + (policy_targets[2035] - policy_targets[2030]) / 5 * (y - 2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Upper Bound (3-phase slowdown)",
        "Year": y,
        "Total_Chargers": int(current_upper),
        "New_Chargers": yearly_add,
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

forecast_df = pd.DataFrame(results)

# --- Step 6: Save results to Excel ---
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "pierce_county_ev_projection_constant_lower.xlsx")
with pd.ExcelWriter(desktop_path, engine="openpyxl") as writer:
    forecast_df.to_excel(writer, sheet_name="Forecast", index=False)

# --- Step 7: Charts ---
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)
plt.axvspan(2025, 2030, color="green", alpha=0.1, label="Phase 1")
plt.axvspan(2030, 2040, color="yellow", alpha=0.1, label="Phase 2")
plt.axvspan(2040, 2050, color="orange", alpha=0.1, label="Phase 3")
plt.xlabel("Year")
plt.ylabel("Total Chargers")
plt.title("Pierce County: EV Charger Projection (Lower Constant vs 3-Phase Upper)")
plt.legend()
plt.grid(True)
plt.tight_layout()
charger_chart = os.path.join(os.path.expanduser("~"), "Desktop", "charger_projection_constant_lower.png")
plt.savefig(charger_chart)
plt.close()

# EV Adoption Rate
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Adoption_Rate"], marker="o", label=sc)
plt.xlabel("Year")
plt.ylabel("EV Adoption Rate")
plt.title("Pierce County: EV Adoption Rate Forecast (2025–2050)")
plt.legend()
plt.grid(True)
plt.tight_layout()
adoption_chart = os.path.join(os.path.expanduser("~"), "Desktop", "ev_adoption_rate_constant_lower.png")
plt.savefig(adoption_chart)
plt.close()

# --- Step 8: Embed charts in Excel ---
wb = load_workbook(desktop_path)
ws = wb.create_sheet(title="Charts")
ws.add_image(Image(charger_chart), "A1")
ws.add_image(Image(adoption_chart), "A40")
wb.save(desktop_path)

print(f"\n✅ Projection complete. Excel and charts saved to: {desktop_path}")
print(f"Final 2050 upper bound chargers: {int(current_upper)}")


Pierce County population (25-59): 448201
Existing Superchargers (from Excel): 1
Existing EVs: 16277
Lower bound chargers (geo coverage, 3-mile radius): 64
Original upper bound (1/2500 residents): 179

--- LOWER BOUND DEBUG ---
Existing chargers (from Excel): 1
Target chargers (3-mile coverage): 64
Total additional chargers needed: 63
Exact average build per year: 2.42
---------------------------

✅ Lower-bound chargers total at 2050: 64 (target: 64)

   Year  Total_Chargers  New_Chargers
0  2025               4             3
1  2026               7             3
2  2027              10             3
3  2028              13             3
4  2029              16             3
5  2030              19             3
6  2031              22             3
7  2032              25             3
8  2033              28             3
9  2034              31             3

✅ Projection complete. Excel and charts saved to: /Users/judycheng/Desktop/pierce_county_ev_projection_constant_lower.xlsx
Fin

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math
from openpyxl import load_workbook
from openpyxl.drawing.image import Image

# --- Step 1: Read Excel files ---
super_df = pd.read_excel("/Users/judycheng/Desktop/supercharger in washington state.xls")
residents_df = pd.read_excel("/Users/judycheng/Desktop/Population 2024 age 25 to 59.xlsx")
ev_df = pd.read_excel("/Users/judycheng/Desktop/coordinates_output.xlsm")

# --- Step 2: Kitsap County data ---
kitsap_super = super_df[(super_df["State"] == "Washington") & (super_df["County"] == "Kitsap County")]
kitsap_residents = residents_df[residents_df["County"] == "Kitsap County"]
kitsap_ev = ev_df[(ev_df["State"] == "WA") & (ev_df["County"] == "Kitsap")]

population_column = "total"
population_25_59 = kitsap_residents[population_column].values[0]
num_chargers = len(kitsap_super)
num_evs = len(kitsap_ev)

print(f"Kitsap County population (25-59): {population_25_59}")
print(f"Existing Superchargers (from Excel): {num_chargers}")
print(f"Existing EVs: {num_evs}")

# --- Step 3: Policy targets ---
policy_targets = {2030: 0.45, 2035: 0.60, 2040: 0.70, 2050: 0.95}

# --- Step 4: Lower and Upper Bound charger targets ---
kitsap_county_area = 566  # sq mi
radius_miles = 5
area_per_charger = np.pi * radius_miles**2  # 78.54 sq mi per charger
lower_chargers = int(np.ceil(kitsap_county_area / area_per_charger))  # geo coverage (5-mile radius)

upper_base_target = population_25_59 / 2500  # 1 per 2500 residents

print(f"Lower bound chargers (geo coverage, 5-mile radius): {lower_chargers}")
print(f"Original upper bound (1/5000 residents): {upper_base_target:.0f}")

# --- Step 5: Build scenarios ---
years = list(range(2025, 2051))
results = []

# --- LOWER BOUND (constant build per year, rounded up) ---
current_lower = num_chargers
total_lower_needed = max(0, lower_chargers - num_chargers)
n_years = len(years)
yearly_build_lower_exact = total_lower_needed / n_years  # float, not int

print("\n--- LOWER BOUND DEBUG ---")
print(f"Existing chargers (from Excel): {num_chargers}")
print(f"Target chargers (5-mile coverage): {lower_chargers}")
print(f"Total additional chargers needed: {total_lower_needed}")
print(f"Exact average build per year: {yearly_build_lower_exact:.2f}")
print("---------------------------\n")

for i, y in enumerate(years):
    # Round up build per year
    new_lower = math.ceil(yearly_build_lower_exact)

    # Adjust final year so total hits target exactly
    if current_lower + new_lower > lower_chargers or i == len(years) - 1:
        new_lower = lower_chargers - current_lower

    current_lower += new_lower

    # EV adoption logic
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 10 * (y - 2025) * 0.5
    elif y <= 2035:
        start_rate = 0.10 + (policy_targets[2030] - 0.10) * 0.5
        effective_adoption = start_rate + (policy_targets[2035] - start_rate) / 5 * (y - 2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Lower Bound (5-mile coverage, constant build)",
        "Year": y,
        "Total_Chargers": int(current_lower),
        "New_Chargers": int(new_lower),
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

print(f"✅ Lower-bound chargers total at 2050: {int(current_lower)} (target: {lower_chargers})\n")

# Preview early years
debug_df = pd.DataFrame([r for r in results if r["Scenario"].startswith("Lower Bound")])
print(debug_df.head(10)[["Year", "Total_Chargers", "New_Chargers"]])

# --- UPPER BOUND (3-phase slowdown) ---
base_build_rate = upper_base_target / n_years
current_upper = num_chargers

for y in years:
    if 2025 <= y <= 2030:
        yearly_add = int(round(base_build_rate * 1.0))
    elif 2031 <= y <= 2040:
        yearly_add = int(round(base_build_rate * 0.6))
    elif 2041 <= y <= 2050:
        yearly_add = int(round(base_build_rate * 0.2))
    else:
        yearly_add = 0

    current_upper += yearly_add

    # EV adoption (upper)
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 5 * (y - 2025)
    elif y <= 2035:
        effective_adoption = policy_targets[2030] + (policy_targets[2035] - policy_targets[2030]) / 5 * (y - 2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Upper Bound (3-phase slowdown)",
        "Year": y,
        "Total_Chargers": int(current_upper),
        "New_Chargers": yearly_add,
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

forecast_df = pd.DataFrame(results)

# --- Step 6: Save results to Excel ---
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "kitsap_county_ev_projection_constant_lower.xlsx")
with pd.ExcelWriter(desktop_path, engine="openpyxl") as writer:
    forecast_df.to_excel(writer, sheet_name="Forecast", index=False)

# --- Step 7: Charts ---
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)
plt.axvspan(2025, 2030, color="green", alpha=0.1, label="Phase 1")
plt.axvspan(2030, 2040, color="yellow", alpha=0.1, label="Phase 2")
plt.axvspan(2040, 2050, color="orange", alpha=0.1, label="Phase 3")
plt.xlabel("Year")
plt.ylabel("Total Chargers")
plt.title("Kitsap County: EV Charger Projection (Lower Constant vs 3-Phase Upper)")
plt.legend()
plt.grid(True)
plt.tight_layout()
charger_chart = os.path.join(os.path.expanduser("~"), "Desktop", "kitsap_charger_projection_constant_lower.png")
plt.savefig(charger_chart)
plt.close()

# EV Adoption Rate
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Adoption_Rate"], marker="o", label=sc)
plt.xlabel("Year")
plt.ylabel("EV Adoption Rate")
plt.title("Kitsap County: EV Adoption Rate Forecast (2025–2050)")
plt.legend()
plt.grid(True)
plt.tight_layout()
adoption_chart = os.path.join(os.path.expanduser("~"), "Desktop", "kitsap_ev_adoption_rate_constant_lower.png")
plt.savefig(adoption_chart)
plt.close()

# --- Step 8: Embed charts in Excel ---
wb = load_workbook(desktop_path)
ws = wb.create_sheet(title="Charts")
ws.add_image(Image(charger_chart), "A1")
ws.add_image(Image(adoption_chart), "A40")
wb.save(desktop_path)

print(f"\n✅ Projection complete. Excel and charts saved to: {desktop_path}")
print(f"Final 2050 upper bound chargers: {int(current_upper)}")


Kitsap County population (25-59): 125820
Existing Superchargers (from Excel): 0
Existing EVs: 6428
Lower bound chargers (geo coverage, 5-mile radius): 8
Original upper bound (1/5000 residents): 50

--- LOWER BOUND DEBUG ---
Existing chargers (from Excel): 0
Target chargers (5-mile coverage): 8
Total additional chargers needed: 8
Exact average build per year: 0.31
---------------------------

✅ Lower-bound chargers total at 2050: 8 (target: 8)

   Year  Total_Chargers  New_Chargers
0  2025               1             1
1  2026               2             1
2  2027               3             1
3  2028               4             1
4  2029               5             1
5  2030               6             1
6  2031               7             1
7  2032               8             1
8  2033               8             0
9  2034               8             0

✅ Projection complete. Excel and charts saved to: /Users/judycheng/Desktop/kitsap_county_ev_projection_constant_lower.xlsx
Final 2050

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
import math

# --- Step 1: Read Excel files ---
super_df = pd.read_excel("/Users/judycheng/Desktop/supercharger in washington state.xls")
residents_df = pd.read_excel("/Users/judycheng/Desktop/Population 2024 age 25 to 59.xlsx")
ev_df = pd.read_excel("/Users/judycheng/Desktop/coordinates_output.xlsm")

# --- Step 2: Chelan County data ---
chelan_super = super_df[(super_df["State"] == "Washington") & (super_df["County"] == "Chelan County")]
chelan_residents = residents_df[residents_df["County"] == "Chelan County"]
chelan_ev = ev_df[(ev_df["State"] == "WA") & (ev_df["County"] == "Chelan")]

population_column = "total"
population_25_59 = chelan_residents[population_column].values[0]
num_chargers = len(chelan_super)  # existing chargers
num_evs = len(chelan_ev)

print(f"Chelan County population (25-59): {population_25_59}")
print(f"Existing Superchargers: {num_chargers}")
print(f"Existing EVs: {num_evs}")

# --- Step 3: Policy targets ---
policy_targets = {2030: 0.45, 2035: 0.60, 2040: 0.70, 2050: 0.95}

# --- Step 4: Lower and Upper Bound charger targets ---
county_area = 2965  # sq mi
radius_miles = 15
lower_chargers = math.ceil(county_area / (3.1416 * radius_miles**2))  # 3 chargers
upper_chargers = math.ceil(population_25_59 / 5000)  

print(f"Lower bound chargers (geo coverage): {lower_chargers}")
print(f"Upper bound chargers (population-based): {upper_chargers}")

# --- Step 5: Build scenarios ---
years = list(range(2025, 2051))
scenarios = {
    "Lower Bound (20-mi radius, front-loaded, whole numbers)": {"target": lower_chargers, "start": num_chargers},
    "Upper Bound (10,000 res/charger)": {"target": upper_chargers, "start": num_chargers}  # <-- start from same as lower
}

results = []

for name, params in scenarios.items():
    target_ch = params["target"]
    current_ch = params["start"]
    
    for y in years:
        # --- Charger build-out ---
        if "Lower" in name:
            if current_ch < target_ch:
                new_ch = 1
                if current_ch + new_ch > target_ch:
                    new_ch = target_ch - current_ch
                current_ch += new_ch
            else:
                new_ch = 0
        else:
            # Upper Bound: start from same as lower bound
            remaining = target_ch - current_ch
            remaining_years = 2050 - y + 1
            if remaining > 0:
                new_ch = math.ceil(remaining / remaining_years)
                if current_ch + new_ch > target_ch:
                    new_ch = target_ch - current_ch
                current_ch += new_ch
            else:
                new_ch = 0

        # --- EV adoption ---
        if "Lower" in name:
            if y <= 2030:
                effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 10 * (y - 2025) * 0.5
            elif y <= 2035:
                start_rate = 0.10 + (policy_targets[2030] - 0.10) * 0.5
                effective_adoption = start_rate + (policy_targets[2035] - start_rate) / 5 * (y - 2030)
            elif y <= 2040:
                effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
            else:
                effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)
        else:
            if y <= 2030:
                effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 5 * (y - 2025)
            elif y <= 2035:
                effective_adoption = policy_targets[2030] + (policy_targets[2035] - policy_targets[2030]) / 5 * (y - 2030)
            elif y <= 2040:
                effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
            else:
                effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)

        evs = int(population_25_59 * effective_adoption)

        results.append({
            "Scenario": name,
            "Year": y,
            "Total_Chargers": int(current_ch),
            "New_Chargers": int(new_ch),
            "EVs": evs,
            "Adoption_Rate": round(effective_adoption, 4)
        })

forecast_df = pd.DataFrame(results)

# --- Step 6: Save results to Excel ---
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "chelan_county_ev_projection_2025_2050.xlsx")
with pd.ExcelWriter(desktop_path, engine="openpyxl") as writer:
    forecast_df.to_excel(writer, sheet_name="Forecast", index=False)

# --- Step 7: Charts ---
# 1️⃣ EV Adoption Rate
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Adoption_Rate"], marker="o", label=sc)
plt.axvspan(2025, 2030, color="orange", alpha=0.1, label="Home Charger Period (Lower Bound)")
plt.xlabel("Year")
plt.ylabel("EV Adoption Rate")
plt.title("EV Adoption Rate by Scenario (Chelan County)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(os.path.expanduser("~"), "Desktop", "chelan_adoption_rate.png"))
plt.close()

# 2️⃣ EV Registrations
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["EVs"], marker="o", label=sc)
plt.xlabel("Year")
plt.ylabel("EV Registrations")
plt.title("EV Registrations by Year (Chelan County)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(os.path.expanduser("~"), "Desktop", "chelan_evs_by_year.png"))
plt.close()

# 3️⃣ Total Chargers
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)
plt.xlabel("Year")
plt.ylabel("Total Chargers")
plt.title("Total Chargers by Year (Chelan County)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(os.path.expanduser("~"), "Desktop", "chelan_total_chargers_by_year.png"))
plt.close()

# --- Step 8: Embed charts in Excel ---
wb = load_workbook(desktop_path)
ws = wb.create_sheet(title="Charts")

img1 = Image(os.path.join(os.path.expanduser("~"), "Desktop", "chelan_adoption_rate.png"))
ws.add_image(img1, "A1")

img2 = Image(os.path.join(os.path.expanduser("~"), "Desktop", "chelan_evs_by_year.png"))
ws.add_image(img2, "A25")

img3 = Image(os.path.join(os.path.expanduser("~"), "Desktop", "chelan_total_chargers_by_year.png"))
ws.add_image(img3, "A49")

wb.save(desktop_path)

print(f"\n✅ Chelan County EV Projection with charts saved to: {desktop_path}")


Chelan County population (25-59): 33879
Existing Superchargers: 3
Existing EVs: 1250
Lower bound chargers (geo coverage): 5
Upper bound chargers (population-based): 7

✅ Chelan County EV Projection with charts saved to: /Users/judycheng/Desktop/chelan_county_ev_projection_2025_2050.xlsx
